In [77]:
filepath = '/home/adityasidharta/git/datascience2018/datasets'
gauge_path = filepath+'/gauge.csv'
radar_path = filepath+'/radar'

In [164]:
from tqdm import tqdm_notebook as tqdm
from scipy.stats import pearsonr

In [83]:
import pandas as pd
import numpy as np
import os

In [84]:
df_gauge = pd.read_csv(gauge_path)

In [141]:
df_result = pd.DataFrame()
train_week = []
for filename in tqdm(os.listdir(radar_path)):
    week = filename.split('.')[0][-2:]
    train_week.append(str(week))
    full_radar_path = os.path.join(radar_path, filename)
    df_radar = pd.read_csv(full_radar_path)
    radar_obs = df_radar.values[:,2:]
    radar_arange = np.arange(0, 34)
    radar_mean = np.matmul(radar_obs, radar_arange) / 2016.
    if 'latlong' not in df_result.columns:
        df_result['latlong'] = df_radar['lat'] * 1000 + df_radar['long']
    assert np.allclose(df_result['latlong'].values, df_radar['lat'] * 1000 + df_radar['long'])
    df_result['mean_week_'+str(week)] = radar_mean

In [142]:
df_result = df_result[sorted(df_result.columns)]

In [143]:
df_result.head()

,latlong,mean_week_13,mean_week_15,mean_week_16,mean_week_17,mean_week_18,mean_week_19,mean_week_20,mean_week_21,mean_week_22,...,mean_week_39,mean_week_41,mean_week_44,mean_week_45,mean_week_46,mean_week_47,mean_week_48,mean_week_49,mean_week_50,mean_week_51
0,0,0.675099,0.382937,0.535714,0.815476,0.248016,0.237599,0.416667,0.908730,0.306052,...,0.193948,0.994544,0.695933,1.006944,1.025298,0.670139,0.461310,0.645337,0.406250,0.157242
1,1,0.714782,0.382440,0.493552,0.841270,0.258433,0.266369,0.450397,0.963790,0.289187,...,0.164683,0.980655,0.738591,1.010417,1.021825,0.635913,0.457341,0.637897,0.339782,0.183532
2,2,0.742063,0.411706,0.519345,0.859127,0.235615,0.260417,0.466270,0.968254,0.324901,...,0.145833,0.958333,0.691468,1.002480,1.012897,0.616071,0.469742,0.611607,0.336806,0.154266
3,3,0.746032,0.380456,0.522321,0.887897,0.247520,0.278274,0.474702,0.954365,0.307540,...,0.157738,0.932044,0.755952,1.075893,1.037202,0.561012,0.494048,0.574405,0.350694,0.157242
4,4,0.745536,0.380456,0.519345,0.860119,0.248016,0.277778,0.472222,0.950893,0.304563,...,0.155258,0.923115,0.771825,1.070933,1.038690,0.553075,0.499008,0.573909,0.352183,0.154266


In [144]:
df_gauge = pd.read_csv(gauge_path)

In [145]:
df_gauge['Year'] = df_gauge['Year'].astype(str)
df_gauge['Week'] = df_gauge['Week'].astype(str)
df_gauge = df_gauge.apply(lambda x: x.fillna(x[2:].mean()),axis=1)

In [146]:
train_week = sorted(train_week)

In [184]:
train_bool = df_gauge['Week'].apply(lambda x : x in train_week)
train_df_gauge = df_gauge[train_bool]

In [185]:
train_df_gauge_array = train_df_gauge.values[:, 2:]

In [186]:
train_df_gauge.head()

,Year,Week,0,1,2,3,4,5,6,7,...,40,41,42,43,44,45,46,47,48,49
12,2017,13,36.8,30.0,48.2,78.6,67.4,45.6,77.6,21.6,...,37.800000,34.0,43.645455,19.4,25.2,26.6,4.8,57.000000,107.000000,54.000000
14,2017,15,43.8,90.4,82.6,13.2,45.8,86.6,34.6,38.4,...,46.000000,99.0,66.773810,66.6,72.9,76.2,82.4,53.600000,21.800000,66.773810
15,2017,16,98.8,134.0,68.2,74.8,77.7,86.4,37.8,102.6,...,97.579487,139.0,97.579487,58.6,69.7,68.2,162.6,97.579487,43.000000,97.579487
16,2017,17,100.2,56.0,82.8,69.2,127.3,84.6,78.0,24.8,...,64.136842,46.2,89.200000,28.0,38.7,112.6,17.8,34.600000,64.136842,32.400000
17,2017,18,2.0,76.0,1.6,1.2,54.5,25.2,30.4,49.8,...,36.884211,24.8,36.884211,29.2,25.5,109.8,71.6,14.000000,36.884211,9.600000


In [187]:
train_df_gauge_array.shape


(31, 50)

In [188]:
df_result['latlong'].values

array([     0,      1,      2, ..., 479477, 479478, 479479])

In [189]:
df_corr_result = pd.DataFrame()
df_corr_result['latlong'] = df_result['latlong'].values
for station in tqdm(range(train_df_gauge_array.shape[1])):
    station_values = train_df_gauge_array[:, 1]
    total_latlong = []
    for idx, latlong in tqdm(df_result.iterrows()):
        latlong_values = latlong.values[1:]
        correlation = np.corrcoef(station_values.tolist(), latlong_values.tolist())[0][1]
        total_latlong.append(correlation)
    df_corr_result['corr_station_'+str(station)] = total_latlong

KeyboardInterrupt: 

174336it [01:24, 2060.51it/s]